# Importowanie bibliotek
W projekcie wykorzystano następujące biblioteki:
- TensorFlow i Keras: Do budowy i trenowania modelu głębokiego uczenia.
- NumPy i Pandas: Do manipulacji danymi i ich wstępnego przetwarzania.
- Scikit-learn: Do skalowania danych oraz podziału na zbiory treningowe i testowe.
- Keras Tuner: Do optymalizacji hiperparametrów modelu.
- Callbacks w Keras: EarlyStopping i ReduceLROnPlateau zapewniają zatrzymanie treningu w odpowiednim momencie i dostosowywanie tempa uczenia.

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras_tuner.tuners import BayesianOptimization
import numpy as np
import tensorflow as tf 
from tensorflow import keras 
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

# Wczytanie i przygotowanie danych
Źródło danych: Wczytano dane z pliku CSV (data.csv).

Filtracja stylów: Wybrano 20 najliczniejszych stylów piwa na podstawie liczby ich wystąpień.

Wybór cech: Uwzględniono numeryczne kolumny, takie jak:
- Size(L) - rozmiar partii piwa.
- OG i FG - gęstość początkowa i końcowa.
- ABV - zawartość alkoholu (%).
- IBU - jednostki goryczki.
- Color - barwa piwa.
- BoilSize, BoilTime, BoilGravity, Efficiency - parametry procesu warzenia.

Przetwarzanie brakujących danych: Brakujące wartości w kolumnach numerycznych zastąpiono średnimi z tych kolumn.

Kodowanie etykiet: Style piwa zakodowano na wartości liczbowe za pomocą LabelEncoder.

Normalizacja danych: Dane wejściowe zostały znormalizowane za pomocą StandardScaler.

Podział na zbiory danych
- Podział danych: Dane podzielono na:
  - Zbiór treningowy (80%): (34216, 10)
  - Zbiór testowy (20%): (8554, 10)
- Konwersja etykiet:
  - Zbiór treningowy: (34216, 20)
  - Zbiór testowy: (8554, 20)

In [2]:
data = pd.read_csv('../data.csv')

top_20_styles = data['Style'].value_counts().nlargest(20).index
filtered_data = data[data['Style'].isin(top_20_styles)]

numeric_columns = ['Size(L)', 'OG', 'FG', 'ABV', 'IBU', 'Color', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = filtered_data[numeric_columns].copy()

X = X.fillna(X.mean())

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(filtered_data['Style'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

num_classes = len(np.unique(y))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("Kształt danych treningowych:", X_train.shape)
print("Kształt danych testowych:", X_test.shape)
print("Kształt etykiet treningowych:", y_train.shape)
print("Kształt etykiet testowych:", y_test.shape)

Kształt danych treningowych: (34216, 10)
Kształt danych testowych: (8554, 10)
Kształt etykiet treningowych: (34216, 20)
Kształt etykiet testowych: (8554, 20)


# Struktura modelu
Zbudowano model sekwencyjny z następującymi warstwami:
- Warstwa wejściowa: Liczba neuronów dynamicznie dobierana z zakresu 128–512, aktywacja ReLU.
- Batch Normalization: Stabilizacja procesu uczenia.
- Dropout: Regularizacja z wartością dynamicznie dobieraną (20–70%).
- Warstwy ukryte:
  - Pierwsza warstwa: Liczba neuronów z zakresu 64–256, aktywacja ReLU.
  - Druga warstwa: Liczba neuronów z zakresu 32–128, aktywacja ReLU.
  - Trzecia warstwa: Liczba neuronów z zakresu 16–64, aktywacja ReLU.
- Warstwa wyjściowa: 20 neuronów (liczba klas) z aktywacją softmax.


Kompilacja modelu
- Optymalizator: Adam z dynamicznie dobieraną szybkością uczenia (0.0001–0.01).
- Funkcja straty: categorical_crossentropy.
- Metryka: accuracy.

In [3]:
units_layer_1_list = [128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512]
units_layer_2_list = [64, 96, 128, 160, 192, 224, 256]
units_layer_3_list = [32, 48, 64, 80, 96, 112, 128]
units_layer_4_list = [16, 32, 48, 64]
dropout_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
learning_rate_list = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2]
batch_size_list = [16, 32, 48, 64, 80, 96, 112, 128]

def build_model(hp):
    model = Sequential()
    
    model.add(Dense(
        units=hp.Choice('units_layer_1', values=units_layer_1_list), 
        activation='relu',
        input_shape=(X_train.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_1', values=dropout_list))) 
    
    model.add(Dense(
        units=hp.Choice('units_layer_2', values=units_layer_2_list), 
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_2', values=dropout_list))) 
    
    model.add(Dense(
        units=hp.Choice('units_layer_3', values=units_layer_3_list),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_3', values=dropout_list))) 
    
    model.add(Dense(
        units=hp.Choice('units_layer_4', values=units_layer_4_list),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_4', values=dropout_list))) 
    
    model.add(Dense(20, activation='softmax'))
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=learning_rate_list)), 
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Proces tuningu hiperparametrów
Metoda tuningu: Bayesian Optimization.

Zakres parametrów:
- Liczba neuronów: Dla każdej warstwy określono zakres możliwych wartości.
- Dropout: Wartości z przedziału 20–70%.
- Learning rate: Wartości z przedziału 0.0001–0.01.
- Batch size: Wartości od 16 do 128.

Liczba prób: 20 (maksymalna liczba konfiguracji).

In [4]:
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='logs/fit',
    project_name='klasyfikacja_20_styli_piwa'
)

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Proces uczenia
Parametry treningu:
- Maksymalna liczba epok: 20.
- Batch size: 32.

Callbacki:
- EarlyStopping: Monitorowanie straty walidacji, zatrzymanie po 5 epokach bez poprawy.
- TensorBoard: Śledzenie wyników w czasie rzeczywistym.


In [5]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,          
    restore_best_weights=True  
)

log_dir = "logs/fit/klasyfikacja_20_styli_piwa" 
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

tuner.search(
    X_train, y_train,
    epochs=20, 
    validation_split=0.2,
    callbacks=[early_stopping, tensorboard_callback],
    batch_size=32 
)

Trial 20 Complete [00h 00m 32s]
val_accuracy: 0.5109584927558899

Best val_accuracy So Far: 0.5168030261993408
Total elapsed time: 00h 10m 29s


In [6]:
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 352)            │         3,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 352)            │         1,408 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 352)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        22,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │         6,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 48)             │         4,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 48)             │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 20)             │           980 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,580 (158.52 KB)

 Trainable params: 39,460 (154.14 KB)

 Non-trainable params: 1,120 (4.38 KB)

# Wyniki
Najlepszy model: Osiągnął dokładność walidacyjną 51.68%.

Finalna dokładność walidacyjna: 51.05%.

Strata walidacyjna: 1.4524.

In [7]:
history = best_model.fit(
    X_train, y_train,
    epochs=20,  
    validation_split=0.2,
    callbacks=[early_stopping, tensorboard_callback],
    batch_size=32  
)

final_val_accuracy = history.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {final_val_accuracy:.2f}")

Epoch 1/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4660 - loss: 1.5988 - val_accuracy: 0.5117 - val_loss: 1.4750
Epoch 2/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4656 - loss: 1.5915 - val_accuracy: 0.5086 - val_loss: 1.4697
Epoch 3/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4675 - loss: 1.6049 - val_accuracy: 0.5117 - val_loss: 1.4656
Epoch 4/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4569 - loss: 1.6127 - val_accuracy: 0.5063 - val_loss: 1.4715
Epoch 5/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4639 - loss: 1.5947 - val_accuracy: 0.5133 - val_loss: 1.4679
Epoch 6/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4610 - loss: 1.5955 - val_accuracy: 0.5118 - val_loss: 1.4640
Epoch 7/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4697 - loss: 1.5866 - val_accuracy: 0.5083 - val_loss: 1.4692
Epoch 8/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4638 - loss: 1.5896 - val_accuracy: 0.

In [11]:
%reload_ext tensorboard
%tensorboard --logdir logs/fit/klasyfikacja_20_styli_piwa

Reusing TensorBoard on port 6007 (pid 7792), started 0:00:08 ago. (Use '!kill 7792' to kill it.)